# BERT LARGE : Facing ResourceExhaustedError

My Drive Storage Current status:


# 254.1 MB of 15 GB used.

# Need your suggestions please



# TRAIN : SET - 1

# VAL : 

> # SET - 2

> # SET - 3

> # test

In [5]:
!pip install -q ktrain

In [6]:
import ktrain
from ktrain import text 

In [7]:
import tensorflow as tf
import numpy as np 
import pandas as pd 

# Load data

In [8]:
set1 = pd.read_csv('/content/drive/My Drive/Tweets/3. Feature Engineering/set1.csv')
set2 = pd.read_csv('/content/drive/My Drive/Tweets/3. Feature Engineering/set2.csv')
set3 = pd.read_csv('/content/drive/My Drive/Tweets/3. Feature Engineering/set3.csv')

test = pd.read_csv('/content/drive/My Drive/Tweets/3. Feature Engineering/tweets_test_features.csv')

In [9]:
set1['target'] = set1['target'].apply(lambda x : 'disaster' if x==1 else 'not_a_disaster')
set2['target'] = set2['target'].apply(lambda x : 'disaster' if x==1 else 'not_a_disaster')

In [10]:
x_train = set1.text.values.tolist()
y_train = set1.target.values.tolist()

x_test = set2.text.values.tolist()
y_test = set2.target.values.tolist()

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# MODEL BUILDING

In [22]:
MODEL_NAME = 'bert-large-uncased'

In [23]:
t = text.Transformer(MODEL_NAME, maxlen=500)

In [24]:
trn = t.preprocess_train(x_train, y_train)
val = t.preprocess_test(x_test, y_test)


preprocessing train...
language: en
train sequence lengths:
	mean : 15
	95percentile : 24
	99percentile : 27


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 15
	95percentile : 24
	99percentile : 27


In [25]:
model = t.get_classifier()



# Train: set- 1
# Val : set- 2

In [26]:
learner = ktrain.get_learner(model, train_data=trn,val_data=val, batch_size=6) #predictor = ktrain.get_predictor(learner.model, preproc=t)

In [27]:
learner.fit_onecycle(lr = 2e-5, epochs = 2 )



begin training using onecycle policy with max lr of 2e-05...
Train for 635 steps, validate for 60 steps
Epoch 1/2
  1/635 [..............................] - ETA: 6:44:06

ResourceExhaustedError: ignored

# F1 SCORE

In [ ]:
learner.validate(class_names = ['Disaster','not_a_disaster'])

# Predict Proba

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

In [ ]:
x2 = set2.text.values
x3 = set3.text.values
test_data = test.text.values

In [ ]:
x2_proba_arr = predictor.predict(x2, return_proba=True)

In [ ]:
x3_proba_arr = predictor.predict(x3, return_proba=True)

In [ ]:
test_proba_arr = predictor.predict(test_data, return_proba=True)

In [ ]:
x2_proba_list = x2_proba_arr.tolist()
x3_proba_list = x3_proba_arr.tolist()
test_proba_list = test_proba_arr.tolist()

In [ ]:
disaster_probability = []
not_a_disaster_probability = []
for i in x2_proba_list:
  disaster_probability.append(i[0])
  not_a_disaster_probability.append(i[1])

x2_df = pd.DataFrame({'Roberta_1': disaster_probability, 'Roberta_0': not_a_disaster_probability})


In [ ]:
disaster_probability = []
not_a_disaster_probability = []
for i in x3_proba_list:
  disaster_probability.append(i[0])
  not_a_disaster_probability.append(i[1])

x3_df = pd.DataFrame({'Roberta_1': disaster_probability, 'Roberta_0': not_a_disaster_probability})

In [ ]:
disaster_probability = []
not_a_disaster_probability = []
for i in test_proba_list:
  disaster_probability.append(i[0])
  not_a_disaster_probability.append(i[1])

test_df = pd.DataFrame({'Roberta_1': disaster_probability, 'Roberta_0': not_a_disaster_probability})

# Save Probas to drive

In [ ]:
x2_df.to_csv('set2_Roberta_proba.csv', index=False) 
!cp set2_Roberta_proba.csv "/content/drive/My Drive/Tweets/5. Predict Probas" 

x3_df.to_csv('set3_Roberta_proba.csv', index=False) 
!cp set3_Roberta_proba.csv "/content/drive/My Drive/Tweets/5. Predict Probas" 

test_df.to_csv('test_data_Roberta_proba.csv', index=False) 
!cp test_data_Roberta_proba.csv "/content/drive/My Drive/Tweets/5. Predict Probas" 

# Sample of the files saved

In [ ]:
x2_df[:2]

In [ ]:
x3_df[:2]

In [ ]:
test_df[:2]